In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
adv_data = pd.read_csv('adv_data.csv')
trad_data = pd.read_csv('trad_data.csv')
sum_data = pd.read_csv('sum_data.csv')
adv_data = adv_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3'])
trad_data = trad_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'])
sum_data = sum_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3', 'Unnamed: 0.4', 'Unnamed: 0.5', 'Unnamed: 0.6'])
adv_data = adv_data.drop_duplicates()
trad_data = trad_data.drop_duplicates()
sum_data = sum_data.drop_duplicates()
trad_data = trad_data.drop(columns=['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN'])

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_4382/2761939097.py:2: DtypeWarning: Columns (3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  trad_data = pd.read_csv('trad_data.csv')


In [4]:
def filter_box_scores(df):
    to_remove = [
        'WH_STATUS',
        'LIVE_PERIOD_TIME_BCAST',
        'NATL_TV_BROADCASTER_ABBREVIATION',
        'NICKNAME',
        'COMMENT',
        'E_OFF_RATING',
        # 'OFF_RATING',
        'E_DEF_RATING',
        'E_NET_RATING',
        'NET_RATING',
        # 'AST_PCT',
        'AST_TOV',
        'AST_RATIO',
        # 'OREB_PCT',
        'DREB_PCT',
        'REB_PCT',
        # 'TM_TOV_PCT',
        'EFG_PCT',
        # 'PIE',
        'FGM',
        'FGA',
        'FG_PCT',
        'FG3M',
        'FG3A',
        'FG3_PCT',
        'FTM',
        'FTA',
        'FT_PCT',
        'OREB',
        # 'DREB',
        'BLK',
        # 'STL',
        # 'TO',
        'PF',
        'PLUS_MINUS',
        'GAMECODE',
        'LIVE_PC_TIME',
        'LIVE_PERIOD',
        'TEAM_CITY',
        'GAME_STATUS_ID',
        'GAME_SEQUENCE',
        'GAME_STATUS_TEXT',
        # 'PACE_PER40',
        'E_USG_PCT',
        'E_PACE',
        'SEASON',
        'PACE',
        # 'POSS',
        # 'TS_PCT',
        # 'USG_PCT',
        'HOME_TEAM_ID',
        'VISITOR_TEAM_ID',
        'REB',
        # 'MIN'
    ]

    for c in to_remove:
        if c in df:
            df = df.drop(c, axis=1)

    return df

def fix_time(df):
    df['YEAR'] = df['GAME_DATE_EST'].str[:4].astype(int)
    df['MONTH'] = df['GAME_DATE_EST'].str[5:7].astype(int)
    df['DAY'] = df['GAME_DATE_EST'].str[8:10].astype(int)
    df = df.drop('GAME_DATE_EST', axis=1)
    return df

def fix_min(df):
    df['SEC'] = df['MIN'].str[-2:].astype(float)
    df['MIN'] = df['MIN'].str[:-3].astype(float)
    df['MIN'] = df['MIN'] + (df['SEC'] / 60)
    df = df.drop(columns=['SEC'])
    return df

def is_home(df):
    df['HOME?'] = (df['TEAM_ID'] == df['HOME_TEAM_ID']).astype(int)
    return df

In [5]:
all_data = pd.merge(adv_data, trad_data, on=['GAME_ID', 'PLAYER_ID'], how='inner')
all_data = pd.merge(all_data, sum_data, on='GAME_ID', how='inner')
starter_data = all_data[all_data['START_POSITION'].isin(['G', 'F', 'C'])]
# starter_data = is_home(starter_data)
# starter_data = filter_box_scores(starter_data)
# starter_data = fix_time(starter_data)
# starter_data = fix_min(starter_data)
# starter_data = starter_data.sort_values(by=['PLAYER_ID', 'YEAR', 'MONTH', 'DAY'])

In [6]:
trad_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 482891 entries, 0 to 484386
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   GAME_ID     482891 non-null  object 
 1   PLAYER_ID   482891 non-null  object 
 2   FGM         401481 non-null  float64
 3   FGA         401481 non-null  float64
 4   FG_PCT      401481 non-null  float64
 5   FG3M        401481 non-null  float64
 6   FG3A        401481 non-null  float64
 7   FG3_PCT     401481 non-null  float64
 8   FTM         401481 non-null  float64
 9   FTA         401481 non-null  float64
 10  FT_PCT      401481 non-null  float64
 11  OREB        401481 non-null  float64
 12  DREB        401481 non-null  float64
 13  REB         401481 non-null  float64
 14  AST         401481 non-null  float64
 15  STL         401481 non-null  float64
 16  BLK         401481 non-null  float64
 17  TO          401481 non-null  float64
 18  PF          401481 non-null  float64
 19  PTS    

In [1195]:
starter_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178635 entries, 404591 to 35678
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   GAME_ID            178635 non-null  object 
 1   TEAM_ID            178635 non-null  int64  
 2   TEAM_ABBREVIATION  178635 non-null  object 
 3   PLAYER_ID          178635 non-null  object 
 4   PLAYER_NAME        178635 non-null  object 
 5   START_POSITION     178635 non-null  object 
 6   OFF_RATING         178635 non-null  float64
 7   DEF_RATING         178635 non-null  float64
 8   AST_PCT            178635 non-null  float64
 9   OREB_PCT           178635 non-null  float64
 10  TM_TOV_PCT         178635 non-null  float64
 11  TS_PCT             178635 non-null  float64
 12  USG_PCT            178635 non-null  float64
 13  PACE_PER40         178635 non-null  float64
 14  POSS               178635 non-null  int64  
 15  PIE                178635 non-null  float64
 16  DRE

In [1207]:
block_size = 30 # amount of games of context we are using for average stats

def context_player(df):
    df['PTS_y'] = df['PTS']

    to_block = [
        'OFF_RATING',
        'DEF_RATING',
        'AST_PCT',
        'OREB_PCT',
        'TM_TOV_PCT',
        'TS_PCT',
        'USG_PCT',
        'PACE_PER40',
        'POSS',
        'PIE',
        'DREB',
        'AST',
        'STL',
        'TO',
        'PTS',
        'MIN'
    ]

    for stat in to_block:
        df[stat] = df[stat].rolling(window=block_size, min_periods=1).mean().shift(-block_size)
    
    return df

starter_context = starter_data.groupby('PLAYER_ID').apply(context_player)
starter_context = starter_context.dropna()

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_27693/2593485627.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  starter_context = starter_data.groupby('PLAYER_ID').apply(context_player)


In [1208]:
def keep_full_games(df):
    df['PC'] = df.groupby('GAME_ID')['GAME_ID'].transform('count')
    df = df[df['PC'] == 10]
    df = df.drop(columns=['PC'])
    return df

starter_context = keep_full_games(starter_context)
starter_context = starter_context.sort_values(by=['GAME_ID', 'TEAM_ID', 'START_POSITION'])

In [1209]:
tor = []
tap = []
tup = []
tpace = []
tpie = []
tto= []
tpts = []

for i in range(len(starter_context)):
    j = i - (i % 5)
    for k in range(j, j+5):
        if k != i:
            tor.append(starter_context['OFF_RATING'].iloc[k])
            tap.append(starter_context['AST_PCT'].iloc[k])
            tup.append(starter_context['USG_PCT'].iloc[k])
            tpace.append(starter_context['PACE_PER40'].iloc[k])
            tpie.append(starter_context['PIE'].iloc[k])
            tto.append(starter_context['TO'].iloc[k])
            tpts.append(starter_context['PTS'].iloc[k])

starter_context['T1_OR'] = tor[::4]
starter_context['T1_AP'] = tap[::4]
starter_context['T1_UP'] = tup[::4]
starter_context['T1_PACE'] = tpace[::4]
starter_context['T1_PIE'] = tpie[::4]
starter_context['T1_TO'] = tto[::4]
starter_context['T1_PTS'] = tpts[::4]
starter_context['T2_OR'] = tor[1::4]
starter_context['T2_AP'] = tap[1::4]
starter_context['T2_UP'] = tup[1::4]
starter_context['T2_PACE'] = tpace[1::4]
starter_context['T2_PIE'] = tpie[1::4]
starter_context['T2_TO'] = tto[1::4]
starter_context['T2_PTS'] = tpts[1::4]
starter_context['T3_OR'] = tor[2::4]
starter_context['T3_AP'] = tap[2::4]
starter_context['T3_UP'] = tup[2::4]
starter_context['T3_PACE'] = tpace[2::4]
starter_context['T3_PIE'] = tpie[2::4]
starter_context['T3_TO'] = tto[2::4]
starter_context['T3_PTS'] = tpts[2::4]
starter_context['T4_OR'] = tor[3::4]
starter_context['T4_AP'] = tap[3::4]
starter_context['T4_UP'] = tup[3::4]
starter_context['T4_PACE'] = tpace[3::4]
starter_context['T4_PIE'] = tpie[3::4]
starter_context['T4_TO'] = tto[3::4]
starter_context['T4_PTS'] = tpts[3::4]

om1 = []
om2 = []
om3 = []
om4 = []
om5 = []
od1 = []
od2 = []
od3 = []
od4 = []
od5 = []
i = 0
while i < len(starter_context):
    if i % 10 == 0:
        for j in range(5):
            om1.append(starter_context['MIN'].iloc[i+5])
            od1.append(starter_context['DEF_RATING'].iloc[i+5])
            om2.append(starter_context['MIN'].iloc[i+6])
            od2.append(starter_context['DEF_RATING'].iloc[i+6])
            om3.append(starter_context['MIN'].iloc[i+7])
            od3.append(starter_context['DEF_RATING'].iloc[i+7])
            om4.append(starter_context['MIN'].iloc[i+8])
            od4.append(starter_context['DEF_RATING'].iloc[i+8])
            om5.append(starter_context['MIN'].iloc[i+9])
            od5.append(starter_context['DEF_RATING'].iloc[i+9])
    else:
        for j in range(5):
            om1.append(starter_context['MIN'].iloc[i-5])
            od1.append(starter_context['DEF_RATING'].iloc[i-5])
            om2.append(starter_context['MIN'].iloc[i-4])
            od2.append(starter_context['DEF_RATING'].iloc[i-4])
            om3.append(starter_context['MIN'].iloc[i-3])
            od3.append(starter_context['DEF_RATING'].iloc[i-3])
            om4.append(starter_context['MIN'].iloc[i-2])
            od4.append(starter_context['DEF_RATING'].iloc[i-2])
            om5.append(starter_context['MIN'].iloc[i-1])
            od5.append(starter_context['DEF_RATING'].iloc[i-1])
    i += 5

starter_context['O1_MIN'] = om1
starter_context['O1_DR'] = od1
starter_context['O2_MIN'] = om2
starter_context['O2_DR'] = od2
starter_context['O3_MIN'] = om3
starter_context['O3_DR'] = od3
starter_context['O4_MIN'] = om4
starter_context['O4_DR'] = od4
starter_context['O5_MIN'] = om5
starter_context['O5_DR'] = od5

In [1210]:
starter_context.to_csv('clean_data.csv')

In [1211]:
stats = pd.read_csv('clean_data.csv')
stats = stats.drop(columns=['Unnamed: 1', 'PLAYER_ID.1'])

In [1212]:
ref_data = pd.read_csv('ref_data.csv')
ref_data = ref_data.drop(columns=['Unnamed: 0', '1', '2', '3'])
ref_data = ref_data.rename(columns={'0': 'REF_ID', '4': 'GAME_ID'})
ref_data = ref_data.sort_values(by='GAME_ID')

In [1213]:
refs = list(set(ref_data['REF_ID']))
rtoi = {r:i for i, r in enumerate(refs)}
ref_data['REF_ID'] = ref_data['REF_ID'].map(rtoi)

In [1214]:
ref_data['group'] = ref_data.groupby('GAME_ID').cumcount()
ref_pivot = ref_data.pivot(index='GAME_ID', columns='group', values='REF_ID').reset_index()
ref_pivot = ref_pivot.drop(columns=[3, 4, 5])
ref_pivot = ref_pivot.dropna()
ref_pivot.columns = ['GAME_ID', 'REF_1', 'REF_2', 'REF_3']
ref_pivot = ref_pivot.sort_values(by=['GAME_ID'])

In [1215]:
with_refs = pd.merge(stats, ref_pivot, on=['GAME_ID'])
with_refs.to_csv('clean_data.csv')

In [696]:
events_info = pd.read_csv('events.csv')
events_info = events_info.drop(columns=['Unnamed: 0'])
events_info = events_info.drop_duplicates()

In [995]:
def convert_time(df):
    df['YEAR'] = df['date'].str[:4].astype(int)
    df['MONTH'] = df['date'].str[5:7].astype(int)
    df['DAY'] = df['date'].str[8:10].astype(int)
    df['HOUR'] = df['date'].str[11:13].astype(int) - 4

    days_in_month = {
        1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30,
        7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31
    }

    def adjust_date(row):
        if row['HOUR'] < 0:
            row['DAY'] -= 1
        if row['DAY'] <= 0:
            row['MONTH'] -= 1
            if row['MONTH'] == 0:
                row['MONTH'] = 12
                row['YEAR'] -= 1
            row['DAY'] = days_in_month[row['MONTH']]
            if row['MONTH'] == 2:
                if (row['YEAR'] % 4 == 0 and row['YEAR'] % 100 != 0) or (row['YEAR'] % 400 == 0):
                    row['DAY'] = 29  # Leap year
        return row

    df = df.apply(adjust_date, axis=1)
    df = df.drop(columns=['date', 'HOUR'])
    return df

In [1049]:
stats = pd.read_csv('clean_data.csv')
stats = stats.drop(columns=['Unnamed: 0'])

In [1050]:
odds_data = pd.read_csv('final_odds.csv')
odds_data = odds_data.drop(columns=['Unnamed: 0'])
odds_data = odds_data.drop_duplicates()
odds_data = convert_time(odds_data)
odds_data = odds_data.drop('event_id', axis=1)
odds_data = odds_data.sort_values(by=['YEAR', 'MONTH', 'DAY'])
odds_data = odds_data.rename(columns={'player': 'PLAYER_NAME'})
stats = stats.sort_values(by=['YEAR', 'MONTH', 'DAY'])

In [1051]:
fixed_odds_data = []
for i in range(len(odds_data)):
    if i % 2 == 0:
        continue
    else:
        new_row = list(odds_data.iloc[i - 1]) + list(odds_data.iloc[i])
        fixed_odds_data.append(new_row)

new_odds_data = pd.DataFrame(fixed_odds_data)
new_odds_data = new_odds_data.rename(columns={
    1: 'PLAYER_NAME',
    2: 'O_ODDS',
    3: 'O_LINE',
    4: 'YEAR',
    5: 'MONTH',
    6: 'DAY',
    9: 'U_ODDS',
    10: 'U_LINE'
    })

new_odds_data = new_odds_data.drop(columns=[0, 7, 8, 11, 12, 13])

In [1052]:
new_data = pd.merge(stats, new_odds_data, on=['YEAR', 'MONTH', 'DAY', 'PLAYER_NAME'], how='inner')
new_data = new_data.drop_duplicates()
new_data.to_csv('clean_odds.csv')

In [1216]:
def filter_box_scores_2(df):
    to_remove = [
        'WH_STATUS',
        'LIVE_PERIOD_TIME_BCAST',
        'NATL_TV_BROADCASTER_ABBREVIATION',
        'NICKNAME',
        'COMMENT',
        # 'E_OFF_RATING',
        # 'OFF_RATING',
        # 'E_DEF_RATING',
        # 'E_NET_RATING',
        # 'NET_RATING',
        # 'AST_PCT',
        # 'AST_TOV',
        # 'AST_RATIO',
        # 'OREB_PCT',
        # 'DREB_PCT',
        # 'REB_PCT',
        # 'TM_TOV_PCT',
        # 'EFG_PCT',
        # 'PIE',
        # 'FGM',
        # 'FGA',
        # 'FG_PCT',
        # 'FG3M',
        # 'FG3A',
        # 'FG3_PCT',
        # 'FTM',
        # 'FTA',
        # 'FT_PCT',
        # 'OREB',
        # 'DREB',
        # 'BLK',
        # 'STL',
        # 'TO',
        # 'PF',
        # 'PLUS_MINUS',
        'GAMECODE',
        'LIVE_PC_TIME',
        'LIVE_PERIOD',
        'TEAM_CITY',
        'GAME_STATUS_ID',
        'GAME_SEQUENCE',
        'GAME_STATUS_TEXT',
        # 'PACE_PER40',
        # 'E_USG_PCT',
        # 'E_PACE',
        'SEASON',
        # 'PACE',
        # 'POSS',
        # 'TS_PCT',
        # 'USG_PCT',
        'HOME_TEAM_ID',
        'VISITOR_TEAM_ID',
        # 'REB',
        # 'GAME_ID',
        # 'TEAM_ID',
        'TEAM_ABBREVIATION',
        # 'PLAYER_ID',
        'PLAYER_NAME',
        # 'START_POSITION',
        'GAME_DATE_EST',
        'HOME?',
        'YEAR',
        'MONTH',
        'DAY'
    ]

    for c in to_remove:
        if c in df:
            df = df.drop(c, axis=1)

    return df

data = all_data[all_data['START_POSITION'].isin(['G', 'F', 'C'])]
data = is_home(data)
data = filter_box_scores_2(data)
# data = fix_time(data)
data = fix_min(data)

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_27693/68233502.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HOME?'] = (df['TEAM_ID'] == df['HOME_TEAM_ID']).astype(int)


In [1217]:
block_size = 30 # amount of games of context we are using for average stats
recent_size = 5 # amount of games of recent context we are using

def context_player_2(df):
    df['PTS_y'] = df['PTS']

    to_block = [
        'PTS',
        'AST',
        'DEF_RATING',
        'E_OFF_RATING',
        'OFF_RATING',
        'E_DEF_RATING',
        'E_NET_RATING',
        'NET_RATING',
        'AST_PCT',
        'AST_TOV',
        'AST_RATIO',
        'OREB_PCT',
        'DREB_PCT',
        'REB_PCT',
        'TM_TOV_PCT',
        'EFG_PCT',
        'PIE',
        'FGM',
        'FGA',
        'FG_PCT',
        'FG3M',
        'FG3A',
        'FG3_PCT',
        'FTM',
        'FTA',
        'FT_PCT',
        'OREB',
        'DREB',
        'BLK',
        'STL',
        'TO',
        'PF',
        'PLUS_MINUS',
        'PACE_PER40',
        'E_PACE',
        'POSS',
        'TS_PCT',
        'REB',
        'USG_PCT',
        'MIN',
        'E_USG_PCT',
        'PACE',
    ]

    to_recent = [
    ]

    for stat in to_block:
        df[stat] = df[stat].rolling(window=block_size, min_periods=1).mean().shift(-block_size)
    
    for stat in to_recent:
        df[stat] = df[stat].rolling(window=recent_size, min_periods=1).mean().shift(-recent_size)

    return df

data_context = data.groupby('PLAYER_ID').apply(context_player_2)
data_context = data_context.dropna()

/var/folders/1j/ycj6sm297qj2d2b1b3pb8k980000gn/T/ipykernel_27693/3379742174.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_context = data.groupby('PLAYER_ID').apply(context_player_2)


In [1218]:
def keep_full_games(df):
    df['PC'] = df.groupby('GAME_ID')['GAME_ID'].transform('count')
    df = df[df['PC'] == 10]
    df = df.drop(columns=['PC'])
    return df

data_context = keep_full_games(data_context)
data_context = data_context.sort_values(by=['GAME_ID', 'TEAM_ID', 'START_POSITION'])

In [1219]:
data_context = data_context.drop(columns=['PLAYER_ID', 'GAME_ID', 'TEAM_ID', 'START_POSITION'])

In [1228]:
om1 = []
om2 = []
om3 = []
om4 = []
om5 = []
od1 = []
od2 = []
od3 = []
od4 = []
od5 = []

s1 = 'TO'
s2 = 'POSS'
i = 0
while i < len(data_context):
    if i % 10 == 0:
        for j in range(5):
            om1.append(data_context[s1].iloc[i+5])
            od1.append(data_context[s2].iloc[i+5])
            om2.append(data_context[s1].iloc[i+6])
            od2.append(data_context[s2].iloc[i+6])
            om3.append(data_context[s1].iloc[i+7])
            od3.append(data_context[s2].iloc[i+7])
            om4.append(data_context[s1].iloc[i+8])
            od4.append(data_context[s2].iloc[i+8])
            om5.append(data_context[s1].iloc[i+9])
            od5.append(data_context[s2].iloc[i+9])
    else:
        for j in range(5):
            om1.append(data_context[s1].iloc[i-5])
            od1.append(data_context[s2].iloc[i-5])
            om2.append(data_context[s1].iloc[i-4])
            od2.append(data_context[s2].iloc[i-4])
            om3.append(data_context[s1].iloc[i-3])
            od3.append(data_context[s2].iloc[i-3])
            om4.append(data_context[s1].iloc[i-2])
            od4.append(data_context[s2].iloc[i-2])
            om5.append(data_context[s1].iloc[i-1])
            od5.append(data_context[s2].iloc[i-1])
    i += 5

data_context['O1_MIN'] = om1
data_context['O1_DR'] = od1
data_context['O2_MIN'] = om2
data_context['O2_DR'] = od2
data_context['O3_MIN'] = om3
data_context['O3_DR'] = od3
data_context['O4_MIN'] = om4
data_context['O4_DR'] = od4
data_context['O5_MIN'] = om5
data_context['O5_DR'] = od5

In [1223]:
data_context.columns

Index(['MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'PTS_y', 'O1_MIN', 'O1_DR', 'O2_MIN', 'O2_DR', 'O3_MIN',
       'O3_DR', 'O4_MIN', 'O4_DR', 'O5_MIN', 'O5_DR'],
      dtype='object')

In [1224]:
# order = ['PTS_y', 'MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
#        'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
#        'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
#        'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
#        'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
#        'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
#        'PLUS_MINUS', 'T1_MIN', 'T2_MIN', 'T3_MIN', 'T4_MIN', 'T1_AST', 'T2_AST', 'T3_AST', 'T4_AST', 'T1_PTS',  
#        'T2_PTS',   'T3_PTS',   'T4_PTS']

order = ['PTS_y','MIN', 'E_OFF_RATING', 'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING',
       'E_NET_RATING', 'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO',
       'OREB_PCT', 'DREB_PCT', 'REB_PCT', 'TM_TOV_PCT', 'EFG_PCT', 'TS_PCT',
       'USG_PCT', 'E_USG_PCT', 'E_PACE', 'PACE', 'PACE_PER40', 'POSS', 'PIE',
       'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS',  'O1_MIN', 'O2_MIN','O3_MIN','O4_MIN','O5_MIN','O1_DR',  'O2_DR', 
       'O3_DR',  'O4_DR',  'O5_DR']


data_context = data_context[order]

In [1229]:
data_context.corr()

,PTS_y,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,...,O1_MIN,O2_MIN,O3_MIN,O4_MIN,O5_MIN,O1_DR,O2_DR,O3_DR,O4_DR,O5_DR
PTS_y,1.000000,0.449617,0.216471,0.232952,0.021125,0.034819,0.149737,0.156173,0.321270,0.269942,...,0.001463,0.004830,-0.005349,0.008957,0.008022,0.017864,0.019413,0.016312,0.040324,0.017861
MIN,0.449617,1.000000,0.197157,0.230955,-0.044218,-0.018932,0.183542,0.195073,0.442973,0.482126,...,0.004616,-0.001426,0.001218,-0.034723,0.022593,-0.000203,-0.017490,-0.005010,-0.012259,-0.001060
E_OFF_RATING,0.216471,0.197157,1.000000,0.989686,0.120351,0.137328,0.674828,0.671482,0.190469,0.246293,...,0.047636,-0.010519,-0.012954,0.020858,-0.012445,0.095228,0.054775,0.076253,0.119631,0.046991
OFF_RATING,0.232952,0.230955,0.989686,1.000000,0.132827,0.152310,0.657669,0.668285,0.205161,0.260412,...,0.047377,-0.010426,-0.010420,0.019738,-0.013208,0.093181,0.052630,0.077681,0.118328,0.044180
E_DEF_RATING,0.021125,-0.044218,0.120351,0.132827,1.000000,0.990674,-0.651394,-0.641673,-0.010368,0.009462,...,0.007652,0.020465,-0.002988,0.042593,-0.036013,0.085518,0.097527,0.082385,0.132647,0.047890
DEF_RATING,0.034819,-0.018932,0.137328,0.152310,0.990674,1.000000,-0.631485,-0.633439,0.004028,0.023386,...,0.010440,0.021210,-0.003636,0.043559,-0.036784,0.089341,0.099021,0.084339,0.134433,0.048551
E_NET_RATING,0.149737,0.183542,0.674828,0.657669,-0.651394,-0.631485,1.000000,0.990214,0.153291,0.181220,...,0.030716,-0.023251,-0.007677,-0.015715,0.017255,0.009214,-0.030625,-0.002962,-0.007159,0.000320
NET_RATING,0.156173,0.195073,0.671482,0.668285,-0.641673,-0.633439,0.990214,1.000000,0.157597,0.186285,...,0.029225,-0.024126,-0.005425,-0.017327,0.017363,0.005709,-0.033323,-0.002661,-0.008542,-0.001945
AST_PCT,0.321270,0.442973,0.190469,0.205161,-0.010368,0.004028,0.153291,0.157597,1.000000,0.913001,...,0.005809,0.000773,0.001633,-0.004540,-0.005088,0.014874,0.005595,0.013400,0.013154,0.001170
AST_TOV,0.269942,0.482126,0.246293,0.260412,0.009462,0.023386,0.181220,0.186285,0.913001,1.000000,...,0.007266,0.002940,0.000749,-0.001800,0.000882,0.021788,0.011238,0.016884,0.025110,0.007508
